# 批量归一化
1. 损失出现在最后，后面的层训练比较快
2. 数据在最底部，导致底部的层训练较慢 & 底部层一变化所有都得跟着边 & 最后的那些层需要重新学习多次 & 导致收敛变慢
3. 如何在学习底部层的时候避免变化顶部层


#
1. 固定小批量里面的均值和方差
<img alt="image.png" height="200" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-29_15.30.15.png" width="1000"/>

2. 然后再做额外的调整（可学习的参数）（Xi+1 似乎是输出）
<img alt="image.png" height="200" src="/Users/loneker/Desktop/d2l/pic/iShot_2024-04-29_15.33.21.png" width="600"/>

# 批量归一化层
1. 可学习的参数是gamma和beta
2. 作用在 a. 全连接层和卷积层输出上，激活函数前     b. 全连接层和卷积层输入上
3. 对于全连接层作用在特征维
4. 对于卷积层作用在通道维（通道维相当于特征）

# 总结
1. 批量归一化固定小批量中的均值和方差，然后学习出适合的偏移和缩放
2. 可以加速收敛速度，但一般不改变模型精度
3. 不能和丢弃法结合使用

从形式上来看，我们计算出 :eqref:`eq_batchnorm`中的$\hat{\boldsymbol{\mu}}_\mathcal{B}$和${\hat{\boldsymbol{\sigma}}_\mathcal{B}}$，如下所示：

$$\begin{aligned} \hat{\boldsymbol{\mu}}_\mathcal{B} &= \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} \mathbf{x},\\
\hat{\boldsymbol{\sigma}}_\mathcal{B}^2 &= \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} (\mathbf{x} - \hat{\boldsymbol{\mu}}_{\mathcal{B}})^2 + \epsilon.\end{aligned}$$

# Batch Norm从零开始实现

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

# moving_mean 和 moving_var 相当于全局的均值和方差, 做推理时使用，是整个数据集的均值和方差，而不是一个小批量的
# momentum 是用来更新moving_mean 和 moving_var的，通常取0.9
# eps通常有个默认值
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum) :
    if not torch.is_grad_enabled():
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)    # 推理是用全局的mean
    else : 
        assert len(X.shape) in (2, 4)   # 要么是2D全连接层要么是4D卷积层
        if len(X.shape) == 2 :
            mean = X.mean(dim = 0)      # 对每一个特征求平均值
            var = ((X - mean) ** 2).mean(dim = 0)
        else :
            mean = X.mean(dim = (0, 2, 3), keepdim = True)          # 卷积要对输出通道数进行批量归一化
            var = ((X - mean) ** 2).mean(dim = (0, 2, 3), keepdim = True)
        X_hat = (X - mean) / torch.sqrt(var + eps)                  # 训练是用当前的mean
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta
    return Y, moving_mean.data, moving_var.data

In [4]:
class BatchNorm(nn.Module):
    # num_features：完全连接层的输出数量或卷积层的输出通道数。
    # num_dims：2表示完全连接层，4表示卷积层
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 非模型参数的变量初始化为0和1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var
        # 复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

In [6]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), BatchNorm(6, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), BatchNorm(16, num_dims=4), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(16*4*4, 120), BatchNorm(120, num_dims=2), nn.Sigmoid(),
    nn.Linear(120, 84), BatchNorm(84, num_dims=2), nn.Sigmoid(),
    nn.Linear(84, 10))

In [ ]:
lr, num_epochs, batch_size = 1.0, 10, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

In [ ]:
net[1].gamma.reshape((-1,)), net[1].beta.reshape((-1,))

# 简洁实现

In [ ]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5), nn.BatchNorm2d(6), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.BatchNorm2d(16), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2), nn.Flatten(),
    nn.Linear(256, 120), nn.BatchNorm1d(120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.BatchNorm1d(84), nn.Sigmoid(),
    nn.Linear(84, 10))

d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())